In [4]:
from scipy import sparse
import numpy as np
from scipy import stats
from IPython.display import display, Markdown, Latex,Math
from bst import bst,constraint,build_linear
import bisect

In [5]:
shape=(2,2)
rvs1 = stats.randint(10,50).rvs
spf=sparse.random(*shape,density=1,format='csr',data_rvs=rvs1)

row_sum=np.array(spf.sum(axis=1,dtype=int))
col_sum=np.array(spf.sum(axis=0,dtype=int))[0]

rvs2 = stats.bernoulli(1).rvs
spr=sparse.random(*shape,density=1,format='csr',data_rvs=rvs2)

In [26]:
constraints=[]
obj= constraint(-1,0,set(),set({0}))
slack=shape[0]*shape[1]+1
for i in range(0,shape[0]):
    cons=constraint(slack,-row_sum[i][0],set(spr.getrow(i).indices + shape[1]*i+1).union(set({0})),set())
    bisect.insort_left(constraints, cons)
    slack=slack+1

In [27]:
spr_col=spr.transpose()    
for i in range(0,shape[1]):
    cons=constraint(slack,col_sum[i],set({0}),set(spr_col.getrow(i).indices*shape[1] + i+1))
    bisect.insort_left(constraints, cons)
    slack=slack+1

In [28]:
def print_tableu_bst(cons,obj):
    display(Math("Max : "+ display(Math(str(obj)))))
    cons.print()
    
def print_tableu_list(cons,obj):
    display(Math("Max \ "+ str(obj)))
    for constraint in cons:
        display(Math(str(constraint)))

In [29]:
print_tableu_list(constraints,obj)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [30]:
spr.toarray()

array([[1., 1.],
       [1., 1.]])

In [31]:
def single_pivot():
    for i in range(0,len(constraints)):
        plausible_leave = constraints[i].n.intersection(obj.p)
        if plausible_leave == set({}):
            continue
        else:
            entering_arc = constraints[i].slack
            leaving_arc = plausible_leave.pop()
            new_constraint=constraint(leaving_arc,constraints[i].rhs,constraints[i].p.difference({entering_arc}),constraints[i].n.difference({leaving_arc}).union({entering_arc}))
            constraints.remove(constraints[i])
            return new_constraint
        
def update(new):
    global obj,constraints
    obj+=new
    constraints = [cons+new for cons in constraints]
    constraints+=[new]
    constraints.sort(key=lambda x: x.rhs)   
    print_tableu_list(constraints,obj)

new=constraint(0,-constraints[0].rhs,constraints[0].n.union({constraints[0].slack}),constraints[0].p.difference({0}))
constraints.remove(constraints[0])
update(new)

while not (obj.p==set({})):
    new=single_pivot()
    update(new)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [32]:
a=set({1,2,3})
a.pop()

1

In [33]:
row_sum

array([[74],
       [43]])

In [34]:
col_sum

array([67, 50])

In [35]:
spf.toarray()

array([[41., 33.],
       [26., 17.]])

In [36]:
np.array(spf.sum(axis=0,dtype=int))

array([[67, 50]])